In [1]:
#Install and import necessary python libraries
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geopy
!pip install geocoder
import pandas as pd
import numpy as np
import requests 
import random
import folium
import geocoder
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



In [138]:
#Scㄐatch district table from website
df = pd.read_html('https://en.wikipedia.org/wiki/Taipei')[8]
df

Map             District  \
                                                  Map             Name[99]   
0   Beitou Shilin Neihu Zhongshan Song-shan Datong...               Beitou   
1   Beitou Shilin Neihu Zhongshan Song-shan Datong...  Daan (Da-an, Da'an)   
2   Beitou Shilin Neihu Zhongshan Song-shan Datong...               Datong   
3   Beitou Shilin Neihu Zhongshan Song-shan Datong...    Nangang (Nankang)   
4   Beitou Shilin Neihu Zhongshan Song-shan Datong...                Neihu   
5   Beitou Shilin Neihu Zhongshan Song-shan Datong...               Shilin   
6   Beitou Shilin Neihu Zhongshan Song-shan Datong...             Songshan   
7   Beitou Shilin Neihu Zhongshan Song-shan Datong...               Wanhua   
8   Beitou Shilin Neihu Zhongshan Song-shan Datong...              Wenshan   
9   Beitou Shilin Neihu Zhongshan Song-shan Datong...                Xinyi   
10  Beitou Shilin Neihu Zhongshan Song-shan Datong...            Zhongshan   
11  Beitou Shilin Neihu Zhongshan Song-shan Datong...           Zhongzheng   

                                                      Population(Jan. 2016)  \
   Chinese[100]      Pinyin   Wade–Giles   Pe̍h-ōe-jī Population(Jan. 2016)   
0           北投區      Běitóu     Pei-t'ou      Pak-tâu                257922   
1           大安區       Dà'ān        Ta-an       Tāi-an                312909   
2           大同區      Dàtóng     Ta-t'ung     Tāi-tông                131029   
3           南港區     Nángǎng     Nan-kang     Lâm-káng                122296   
4           內湖區       Nèihú       Nei-hu       Lāi-ô͘                287726   
5           士林區      Shìlín     Shih-lin       Sū-lîm                290682   
6           松山區    Sōngshān    Sung-shan    Siông-san                209689   
7           萬華區      Wànhuá      Wan-hua     Báng-kah                194314   
8           文山區     Wénshān     Wen-shan      Bûn-san                275433   
9           信義區       Xìnyì      Hsin-yi       Sìn-gī                229139   
10          中山區   Zhōngshān   Chung-shan    Tiong-san                231286   
11          中正區  Zhōngzhèng  Chung-cheng  Tiong-chèng                162549   

   Area(km2) Postalcode  
   Area(km2) Postalcode  
0    56.8216        112  
1    11.3614        106  
2     5.6815        103  
3    21.8424        115  
4    31.5787        114  
5    62.3682        111  
6     9.2878        105  
7     8.8522        108  
8    31.5090        116  
9    11.2077        110  
10   13.6821        104  
11    7.6071        100

In [139]:
#Data Clean and calculate population density
df2 = df.iloc[:, [1,2,6,7,8]]
df2.set_axis(['District', 'Chinesename','Population', 'Area(km2)', 'Postalcode'], axis=1, inplace=True)
df2['Population Density(/(km2))'] = df2['Population']/df2['Area(km2)']
df2 = df2[['Postalcode','District','Chinesename', 'Population Density(/(km2))']]
df2

<ipython-input-139-b9c47905ed7c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Population Density(/(km2))'] = df2['Population']/df2['Area(km2)']


,Postalcode,District,Chinesename,Population Density(/(km2))
0,112,Beitou,北投區,4539.154124
1,106,"Daan (Da-an, Da'an)",大安區,27541.412150
2,103,Datong,大同區,23062.395494
3,115,Nangang (Nankang),南港區,5599.018423
4,114,Neihu,內湖區,9111.394706
5,111,Shilin,士林區,4660.740570
6,105,Songshan,松山區,22576.821206
7,108,Wanhua,萬華區,21950.927453
8,116,Wenshan,文山區,8741.407217
9,110,Xinyi,信義區,20444.783497


In [140]:
#Get the latitude and longitude and incorporate them with district name
geolocator = Nominatim(user_agent="Taipei_Finder")

df2['Dist_Coord'] = df2['Chinesename'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df2[['Latitude','Longitude']] = df2['Dist_Coord'].apply(pd.Series)
df2.drop(['Dist_Coord'],axis=1,inplace=True)
df2

,Postalcode,District,Chinesename,Population Density(/(km2)),Latitude,Longitude
0,112,Beitou,北投區,4539.154124,25.132419,121.501379
1,106,"Daan (Da-an, Da'an)",大安區,27541.412150,25.026515,121.534395
2,103,Datong,大同區,23062.395494,46.099463,124.697919
3,115,Nangang (Nankang),南港區,5599.018423,25.054578,121.606600
4,114,Neihu,內湖區,9111.394706,25.069664,121.588998
5,111,Shilin,士林區,4660.740570,25.091840,121.524207
6,105,Songshan,松山區,22576.821206,25.049885,121.577272
7,108,Wanhua,萬華區,21950.927453,25.031933,121.499332
8,116,Wenshan,文山區,8741.407217,24.989786,121.570458
9,110,Xinyi,信義區,20444.783497,25.033345,121.566896


In [86]:
#Let's see how these 12 distrincts distributed in Taipei
map_Taipei = folium.Map(location=[25.04776,121.53185],zoom_start=12)

for lat,lng,dist in zip(df2['Latitude'],df2['Longitude'],df2['District']):
    label = '{}'.format(dist)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=10,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3386cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Taipei)
map_Taipei

In [79]:
CLIENT_ID='WV5KSDIUGI44I10EGECXXYIMW3NEI05HFA5CJOB02XHXIFJF'
CLIENT_SECRET = 'HP1ZNPMJFJRSQ4AH4UOIUHJD3XH4TYVM1RJ2JCUWDHSBPA1E'
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20210128',
    "section": "coffee",
    "near": "Taipei, Taiwan",
    "radius": 100000,
    "limit": 100000}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)


In [141]:
d = data.json()["response"]
items = d["groups"][0]["items"]
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    #print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
Taipei_Coffee_df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
Taipei_Coffee_df=Taipei_Coffee_df[Taipei_Coffee_df["shortname"].isin(['Café','Coffee Shop'])]
Taipei_Coffee_df=Taipei_Coffee_df[Taipei_Coffee_df["postalcode"].isin(np.array(df2.iloc[:,0].values).astype(str))]
print("Total number of coffee shop found:" + str(len(Taipei_Coffee_df)))

Total number of coffee shop found:50


In [149]:
#Let's see the distribution of coffe shops in Taipei with labeled dirstricts
import folium.plugins as plugins
from matplotlib import cm
import folium
map_Taipei_Coffee = folium.Map(location=[25.04776,121.53185],zoom_start=12)



for lat,lng,dist in zip(df2['Latitude'],df2['Longitude'],df2['District']):
    label = '{}'.format(dist)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=10,
    popup=label,
    color='Blue',
    fill=True,
    fill_color='Blue',
    fill_opacity=0.1,
    parse_html=False).add_to(map_Taipei_Coffee)
    
    
for (j, row) in Taipei_Coffee_df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=1,
            popup=label,
            color='green',
         
            parse_html=False).add_to(map_Taipei_Coffee)
    
hm_data = Taipei_Coffee_df[["lat", "lng"]].to_numpy().tolist()
map_Taipei_Coffee.add_child(plugins.HeatMap(hm_data))    
    
    
    
    
map_Taipei_Coffee